In [5]:
from model.grid import generateGrid
from model.propabilities import calculateTransitionPropabilitiesForAllPorfolios, select_probabilities_for_chosen_strategies
from model.investment_planner import get_portfolios_strategies, get_optimal_strategies_for_T
import numpy as np
import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
portfolios = np.array([[0.0526, 0.037 ],
       [0.0552, 0.0396],
       [0.0577, 0.0463],
       [0.0603, 0.0556],
       [0.0629, 0.0664],
       [0.0655, 0.0781],
       [0.068 , 0.0904],
       [0.0706, 0.1031],
       [0.0732, 0.1159],
       [0.0757, 0.129 ],
       [0.0783, 0.1421],
       [0.0809, 0.1554],
       [0.0834, 0.1687],
       [0.086 , 0.1821],
       [0.0886, 0.1955]])

In [7]:
W0 = 100
iMax = 401
T = 12
#infusions = [0,5,5,5,5,5,0]
infusions = np.zeros(T)
goals = [None,None,None,None,None,None,None,None,None,None,np.array([[200,1000]]),None]
goal_max_costs = [0,0,0,0,0,0,0,0,0,0,200,0]

#infusions = [ 0]
#goals = [20]
#

In [13]:
grid = generateGrid(W0,iMax,infusions,goal_max_costs,portfolios[0,0],portfolios[0,1],portfolios[-1,0],portfolios[-1,1])

goals_strategies = np.zeros((T,iMax))
portfolios_strategies = np.zeros((T,iMax))
probabilities = np.zeros((T,iMax,iMax))
V = np.zeros((T,iMax))
selected_propabilities = np.zeros((T,iMax,iMax))

for t in range(T-2,-1,-1):
    probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,grid[t],grid[t+1],infusions[t],1)
    result = get_optimal_strategies_for_T(goals[t],grid[t], probabilities, V[t+1])  
    goals_strategies[t] = result.goals_strategies
    portfolios_strategies[t] = result.porfolios_strategies
    V[t] = result.values
    goals_costs = goals[t][:,0] if goals[t] is not None else None
    selected_propabilities[t] = select_probabilities_for_chosen_strategies(probabilities,result.porfolios_strategies,result.goals_strategies,goals_costs,grid[t])
    

In [ ]:
arr = np.array([[[0.367, 0.275, 0.225, 0.133],
                [0.287, 0.264, 0.247, 0.202],
                [0.017, 0.105, 0.244, 0.634],
                [0.001, 0.017, 0.075, 0.908]
                ],
                [[0.367, 0.275, 0.225, 0.133],
                [0.287, 0.264, 0.247, 0.202],
                [0.017, 0.105, 0.244, 0.634],
                [0.001, 0.017, 0.075, 0.908]
                ]])

p = np.zeros(4)

for i in range (4):
    p[i] = arr[1,:,i] @ arr[0,i]

print(arr[0,0,0]*arr[1,0,0])
print(arr[0,0,1]*arr[1,1,0])
print(arr[0,0,2]*arr[1,2,0])
print(arr[0,0,3]*arr[1,3,0])

arr[0,0,0]*arr[1,0,0]+arr[0,0,1]*arr[1,1,0]+arr[0,0,2]*arr[1,2,0]+arr[0,0,3]*arr[1,3,0]
p


0.134689
0.078925
0.0038250000000000003
0.000133


array([0.217572, 0.17799 , 0.136846, 0.875581])

In [ ]:
df = pd.DataFrame(portfolios_strategies.T)
df.insert(0,'W',grid[1].T)
#df.set_index('W')
#sns.heatmap(df.iloc[230:251])
short = df.iloc[200:281]


TODO:
* test get_portfolios_strategies when vt1 for all i
* dodać wyświetlanie portfolios strategies i goal strategies a grid wealth values (data frame)
* wyświetlić z multiple goals

In [ ]:
arr = np.array([[[0.367, 0.275, 0.225, 0.133],
                      [0.287, 0.264, 0.247, 0.202],
                      [0.017, 0.105, 0.244, 0.634],
                      [0.001, 0.017, 0.075, 0.908]
                      ]])  

print(np.round(arr[0,0]@arr[0,:,0],4))
print(np.round(arr[0,0]@arr[0,:,1],4))
print(np.round(arr[0,0]@arr[0,:,2],4))
print(np.round(arr[0,0]@arr[0,:,3],4))

0.2176
0.1994
0.2154
0.3678


In [ ]:
goals = [None,np.array([(100,150),(200,300)])]
t = 0
#goals_costs = goals[t][:,0] if goals[t] is not None else None
goals_costs = goals[t][:,0] if goals[t] is not None else None
goals_costs